In [6]:
from vertexai.preview.language_models import ChatModel, InputOutputTextPair, ChatSession
from google.cloud import aiplatform
from google.cloud import texttospeech
from google.cloud import translate_v2 as translate
from pydub import AudioSegment
from pydub.playback import play
import time
import keyboard
import pyaudio
import numpy as np
from scipy.io import wavfile
from playsound import playsound

In [7]:
thai_context = '''
คุณอยู่ในมหาวิทยาลัยกรุงเทพในวัน openhouse ภารกิจของคุณคือการตอบคำถามเกี่ยวกับมหาวิทยาลัยกรุงเทพโดยเฉพาะคำถามเกี่ยวกับคณะวิศวกรรมศาสตร์ และคุณต้องกระตุ้นให้นักเรียนเข้าเรียนในคณะวิศวกรรมศาสตร์ ข้อควาามต่อไปนี้คือสิ่งที่คุณต้องรู้ :

ตลอดระยะเวลากว่าครึ่งศตวรรษ มหาวิทยาลัยกรุงเทพเป็นผู้นำสถานศึกษาเอกชนชั้นนำของประเทศ ที่ผลิตบัณฑิตที่มีคุณภาพ มีความคิดสร้างสรรค์ และจิตวิญญาณผู้ประกอบการ โดยให้ความสำคัญกับการออกแบบการเรียนการสอนแบบใหม่ที่ผสมผสานความคิดสร้างสรรค์และเทคโนโลยีด้วยหลักสูตรที่ตอบโจทย์อนาคต มีสิ่งแวดล้อมที่สร้างสรรค์ทั้งอาคารเรียนที่โดดเด่นด้านสถาปัตยกรรม ศูนย์การเรียนรู้ ควบคู่กับห้องปฏิบัติการและอุปกรณ์ที่ทันสมัย เพื่อเสริมทักษะการเรียนรู้ของนักศึกษาอย่างเต็มศักยภาพ 
มหาวิทยาลัยกรุงเทพ เป็นมหาวิทยาลัยเอกชนแห่งแรกที่มีขนาดใหญที่สุดใประเทศไทย ก่อตั้งเมื่อวันที่ 25 ธันวาคม 2505 โดยมีอาจารย์สุรัตน์ และอาจารย์ปองทิพย์ โอสถานุเคราะห์ เป็นผู้ก่อตั้ง เดิมชื่อว่า “โรงเรียนไทยเทคนิค” ต่อมาเปลี่ยนชื่อเป็น “วิทยาลัยไทยเทคนิค” และเพื่อความเหมาะสมที่จะเป็นสถาบันอุดมศึกษาเอกชนที่เปิดสอนระดับปริญญาตรี จึงเปลี่ยนชื่อเป็น “วิทยาลัยกรุงเทพ” (Bangkok College) ในปี พ.ศ. 2508 ต่อมาเมื่อวันที่ 24 ตุลาคม 2527 ทบวงมหาวิทยาลัยได้อนุมัติให้เปลี่ยนสถานภาพเป็น “มหาวิทยาลัยกรุงเทพ” (Bangkok University)
ปรัชญาของมหาวิทยาลัยกรุงเทพ คือ บัณฑิตต้องมีความรู้คู่ความดี มีความคิดสร้างสรรค์ และมีจิตวิญญาณผู้ประกอบการที่สามารถก้าวทันการเปลี่ยนแปลงของโลก และดำรงชีวิตอย่างมีความสุข
ปณิธานของมหาวิทยาลัยกรุงเทพ คือ มหาวิทยาลัยกรุงเทพมุ่งพัฒนาบัณฑิตที่มีคุณภาพ มีคุณธรรม จริยธรรม มีความเป็นผู้นำเชิงสร้างสรรค์ มีจิตวิญญาณของการเป็นผู้ประกอบการที่เป็นมืออาชีพด้านวิชาการ และทักษะในทางปฏิบัติด้วยความรับผิดชอบต่อสังคม รวมทั้งเป็นสถาบันในระดับสากลที่เป็นศูนย์รวมการวิจัย ซึ่งมุ่งสร้างความร่วมมือกับสถาบันการศึกษาและองค์กรทั้งในประเทศและต่างประเทศ เพื่อประโยชน์ในการพัฒนาคุณภาพการเรียนการสอนและการให้บริการทางวิชาการแก่สังคม

มหาวิทยาลัยกรุงเทพตั้งอยู่ ณ เลขที่ 9/1 หมู่ที่ 5 ถนนพหลโยธิน ตำบลคลองหนึ่ง อำเภอคลองหลวง จังหวัดปทุมธานี มีพื้นที่ 441 ไร่ 1 งาน 67 ตารางวา เป็นสถานที่ดำเนินการเรียนการสอนของนักศึกษาทุกคณะ แบ่งพื้นที่เป็นโซน A-D ประกอบด้วยอาคาร 47 หลัง มีพื้นที่ใช้สอยอาคารรวม 150,625 ตารางเมตร ประกอบด้วย อาคารเรียน 11 อาคาร อาคารปฏิบัติการ 6 อาคาร โรงอาหาร 4 อาคาร และอาคารอื่นๆ 26 อาคาร แบ่งเป็นห้องบรรยาย 137 ห้อง มีพื้นที่ใช้สอย 17,313 ตารางเมตร ห้องปฏิบัติการ 220 ห้อง มีพื้นที่ใช้สอย 22,892 ตารางเมตร เช่น โซน A มีอาคาร BU Diamond อาคาร Tourism Tower อาคารเรียน อาคาร Imagine Village อาคารบริการ และศูนย์กิจกรรมนักศึกษา (SAC) โซน B มีอาคารเรียนและอาคารฝึกงานของคณะศิลปกรรมศาสตร์ อาคารฝึกงานของคณะสถาปัตยกรรมศาสตร์ อาคารคณะวิศวกรรมศาสตร์ 
พิพิธภัณฑ์สถานเครื่องถ้วยเอเชียตะวันออกเฉียงใต้ อาคารหอสมุดสุรัตน์ โอสถานุเคราะห์ อาคารนิเทศศาสตร์คอมเพล็กซ์ Center for Cinematic and Digital Arts และสำนักพิมพ์มหาวิทยาลัยกรุงเทพ และโซน D อาคารรับรองและอัฒจันทร์สนามเทนนิส ห้องประชุมและห้องสัมมนา 41 ห้อง มีพื้นที่ใช้สอย 5,107 ตารางเมตร ห้องฝ่ายบริหารและห้องทำงานผู้สอน จำนวน 77 ห้องมีพื้นที่ใช้สอย 11,299 ตารางเมตร ห้องพักผ่อนนักศึกษา และห้องอื่นๆ จำนวน 121 ห้อง มีพื้นที่ใช้สอย 9,573 ตารางเมตร และโครงการ Imagine Village มีพื้นที่ใช้สอยประมาณ 13,671 ตารางเมตร ได้รับการรับรองมาตรฐาน ISO14001 ทั้งระบบจากสถาบันสิ่งแวดล้อมไทย ซึ่งเป็นมหาวิทยาลัยเอกชนแห่งแรกในประเทศไทยที่ได้รับการรับรองมาตรฐานทั้งระบบ
นอกจากนี้ มหาวิทยาลัยมีสนามฟุตบอล สนามกีฬากลางแจ้ง ศาลาที่พักรอบมหาวิทยาลัย ร้านค้าย่อย สวนวรรณคดี สวนพันธุ์ไม้ประจำสถาบันฯ สวนสุขภาพ ลานจอดรถ 1-7 ความจุ 1,048 คัน เป็นต้น
สถานที่ต่างๆในมหาวิทยาลัยกรุงเทพ เช่น 
หอสมุด : เปิดโลกความรู้ด้วยหนังสือภาษาไทยและต่างประเทศมากกว่า 300,000 เล่ม วารสารกว่า 80,000 เล่ม สื่อโสตทัศนวัสดุกว่า 15,000 รายการ และมีสิ่งอำนวยความสะดวกต่างๆ เช่นคอมพิวเตอร์ boardgames และห้องดูหนัง
IMAGINE Lounge : เปลี่ยนความสนุกให้เป็นความสร้างสรรค์ ที่ IMAGINE Lounge Student Lounge ที่ดีที่สุด เพื่อความคิดสร้างสรรค์สมบูรณ์แบบ และพื้นที่อิสระสำหรับนักศึกษาในการต่อยอดทางความคิดสร้างสรรค์
พิพิธภัณฑสถานเครื่องถ้วยเอเชียตะวันออกเฉียงใต้ : เราให้มากกว่าความรู้ในห้องเรียน เพราะเรามีพิพิธภัณฑสถานเครื่องถ้วยเอเชียตะวันออกเฉียงใต้ มีลักษณะเป็นอาคารชั้นเดียวตั้งอยู่ชั้นใต้ดิน อยู่ด้านหน้าห้องสมุดสุรัตน์ โอสถานุเคราะห์
ศูนย์กิจกรรมนักศึกษามหาวิทยาลัยกรุงเทพ หรือ SAC (Student Activity Center) : อาคารกิจกรรมระดับรางวัลชนะเลิศด้านการออกแบบ และนิตยสารดีไซน์ชื่อดังของอิตาลี Domus จัดให้เป็น Best School Buildings ระดับโลก มีพื้นที่ 2,200 ตารางเมตร สร้างไว้รองรับกิจกรรมของนักศึกษา
หอศิลปมหาวิทยาลัยกรุงเทพ : พื้นที่สร้างแรงบันดาลใจจากศิลปะร่วมสมัย ค้นหาไอเดียและความคิดสร้างสรรค์ใหม่ๆ ได้อย่างไม่จำกัด หอศิลปฯ จัดนิทรรศการศิลปะของศิลปินทั้งไทยและต่างประเทศ ให้นักศึกษาและบุคคลทั่วไปเข้าชมได้
Imagine Village : Creative Mall ยูนิเวอร์ซิตี้มอลล์แนวใหม่ หมู่บ้านร้านค้าแหล่งกำเนิดนักคิดสร้างสรรค์ ที่เชื่อมโยงการศึกษากับชุมชนไว้รวมกัน พร้อมบริการครบครัน
ศูนย์สร้างสรรค์ มหาวิทยาลัยกรุงเทพ หรือ BUCC (BANGKOK UNIVERSITY CREATIVITY CENTER) : เพราะความคิดสร้างสรรค์คือรากฐานของทุกความสำเร็จ เรามีหน่วยงานหรือพื้นที่จัดเวิร์คช็อปสำหรับนักศึกษาที่สนใจกิจกรรมสร้างสรรค์ผลักดันนักศึกษาที่ได้รับทุนและต่อยอดความคิดสร้างสรรค์ของตัวเองที่นี่
นอกจากนี้มหาวิทยาลัยกรุงเทพยังมีศูนย์กีฬาในร่ม และสนามกีฬาให้บริการ อาทิ สนามฟุตบอลขนาดมาตรฐาน สนามกรีฑา สนามบาสเกตบอล สนามเซปักตะกร้อ สนามวอลเลย์บอล สนามเทนนิส สนามเปตอง และสวนสุขภาพ เป็นต้น
มหาวิทยาลัยกรุงเทพมี โรงอาหาร 3 ที่  โรงอาหาร A3 โรงอาหารกลาง และ โรงอาหารตึกนิเทศ

มหาวิทยาลัยมี 12 คณะที่เปิดสอนหลักสูตรปริญญาตรี ได้แก่ บริหารธุรกิจ บัญชี นิเทศศาสตร์ ดิจิทัลมีเดียและภาพยนตร์ มนุษยศาสตร์และการจัดการการท่องเที่ยว กฎหมาย เศรษฐศาสตร์และการลงทุน เทคโนโลยีสารสนเทศและนวัตกรรม ศิลปกรรมศาสตร์ วิศวกรรมศาสตร์ สถาปัตยกรรมศาสตร์ และคณะการสร้างเจ้าของธุรกิจและการบริหารกิจการ นอกจากนี้ยังมี Bangkok University International (BU International) ที่ยังเปิดสอนหลักสูตรต่างๆ โดยใช้ภาษาอังกฤษเป็นสื่อกลางการเรียนการสอน
คณะวิศวกรรมศาสตร์ของมหาวิทยาลัยกรุงเทพจะ เตรียมความพร้อมวิชาชีพวิศวกร เน้นทั้งทฤษฎี การปฏิบัติ และสร้างงานจริงด้วยห้องปฏิบัติการครบวงจร เรียนรู้งานทั้ง Hardware และ Software รวมถึงเทคโนโลยีที่เกี่ยวข้อง เช่น IoT, AI & Machine Learning และก้าวทันข้อมูลยุคใหม่กับ Big Data ครอบคลุมในส่วนของ Programmer, IT, Smart System, Robotics & Automation หลักสูตรโดดเด่น สร้างทักษะที่แตกต่างจากหลักสูตรวิศวกรรมศาสตร์ทั่วไป ได้ทักษะที่เอาไปใช้ได้จริง ได้ทำงานร่วมกับนักวิจัยนานาชาติ เพิ่มความเป็นสากลให้กับนักศึกษา มีโอกาสได้ใบประกาศนียบัตรวิชาชีพ รองรับ Cisco และ Mircrosoft Certificate  นอกจากนี้หลักสูตรยังได้ผ่านการรับรองมาตรฐานการศึกษาจากสํานักงานคณะกรรมการการอุดมศึกษา และสภาวิศวกร พร้อมตอบโจทย์การพัฒนาเทคโนโลยีในปัจจุบันและอนาคต

ในระดับปริญญาตรีของมหาวิทยาลัยกรุงเทพมี 4 หลักสูตรวิศวกรรมศาสตร์ ได้แก่ วิศวกรรมคอมพิวเตอร์และหุ่นยนต์ วิศวกรรมปัญญาประดิษฐ์และวิทยาการข้อมูล วิศวกรรมมัลติมีเดียและเอ็นเตอร์เทนเมนท์ และวิศวกรรมไฟฟ้า
วิศวกรรมคอมพิวเตอร์และหุ่นยนต์ : มหาวิทยาลัยกรุงเทพได้ตระหนักถึงความสำคัญที่เทคโนโลยีคอมพิวเตอร์ได้เข้ามามีบทบาทในการทำงานและชีวิตประจำวันของมนุษย์มากขึ้น การทำงานลักษณะสมัยใหม่มีการนำเทคโนโลยีคอมพิวเตอร์ไปช่วยงานด้านต่างๆ เช่น การจัดการเอกสารและข้อมูลในระบบสำนักงาน ช่วยควบคุมการผลิตในโรงงานอุตสาหกรรม ตลอดจนการดำรงชีวิตประจำวันที่เกี่ยวข้องกับเครื่องมือเครื่องใช้ที่เป็นระบบอัตโนมัติ และมีส่วนประกอบของคอมพิวเตอร์ 
ดังนั้น ประเทศไทยจึงขาดแคลนบุคลากรที่มีความรู้ความสามารถทางด้านวิศวกรรมคอมพิวเตอร์และหุ่นยนต์ เพื่อสนองตอบความต้องการบุคลากรในด้านนี้ มหาวิทยาลัยกรุงเทพเปิดสอนหลักสูตรวิศวกรรมศาสตร์ ภาควิชาวิชาวิศวกรรมคอมพิวเตอร์และหุ่นยนต์ขึ้นหลักสูตรวิศวกรรมคอมพิวเตอร์และหุ่นยนต์  มีตลาดงานกว้าง ครอบคลุมในส่วนของ Programmer, IT, Smart System, Robotics & Automation เรียนรู้งานทั้ง Hardware และ Software ทางด้านคอมพิวเตอร์และหุ่นยนต์ รวมถึงเทคโนโลยีที่เกี่ยวข้อง เช่น IoT, AI & Machine Learning และก้าวทันข้อมูลยุคใหม่กับ Big Data โดยมีความมุ่งหวังที่จะผลิตบัณฑิต ที่มีความรู้ทักษะและความสามารถในวิชาชีพอย่างสมบูรณ์ เพียบพร้อมด้วยคุณธรรมจริยธรรมที่จะเป็นกำลังคนที่มีค่าของประเทศ
ค่าเล่าเรียนทั้งหมดสำหรับหลักสูตรวิศวกรรมคอมพิวเตอร์และหุ่นยนต์คือ 375,380 บาท
วิศวกรรมปัญญาประดิษฐ์และวิทยาการข้อมูล :เป็นหลักสูตรแรกและหลักสูตรเดียวในไทย ที่เน้นการพัฒนาอุตสาหกรรมข้อมูลและปัญญาประดิษฐ์ ผสานองค์ความรู้ที่จำเป็นแห่งยุครอบด้าน ทั้งทักษะปัญญาประดิษฐ์ วิทยาการข้อมูล เสริมสร้างความรู้ด้านธุรกิจ ด้วยกระบวนการบ่มเพาะที่เน้นการเรียนการสอนรูปแบบใหม่ ให้นักศึกษาได้เรียนรู้เทคโนโลยีใหม่ๆ กับผู้เชี่ยวชาญที่มีประสบการณ์และมีผลงานเด่นที่สนับสนุนหลักสูตร ใช้โจทย์จริงในโลกธุรกิจมาสอนในรายวิชา ทั้งกิจกรรมเสริมหลักสูตร โครงการดูงานหรือสัมมนาพิเศษ เพื่อผลิตวิศวกรข้อมูลและปัญญาประดิษฐ์ให้มีความเชี่ยวชาญทั้งด้านวิชาการและวิชาชีพและพร้อมที่จะปฏิบัติงานได้ทันที มีศักยภาพในการประยุกต์ความรู้ให้เหมาะกับลักษณะงานและอาชีพใหม่ๆ ที่เกิดขึ้นในอนาคต ตอบโจทย์ตลาดงานด้านวิศวกรรมปัญญาประดิษฐ์และวิทยาการข้อมูลที่ต้องการบุคลากรที่มีคุณภาพเป็นจำนวนมาก พร้อมส่งเสริมความเป็นผู้ประกอบการสร้างโอกาสสำหรับแข่งขันทั้งในประเทศไทยและต่างประเทศ 
ค่าเล่าเรียนทั้งหมดสำหรับหลักสูตรวิศวกรรมปัญญาประดิษฐ์และวิทยาการข้อมูลคือ 372,280 บาท
นักศึกษาที่เลือกเรียนสาขานี้จะได้ศึกษาถึงการนำองค์ความรู้ของการผลิตสื่อมัลติมีเดียและบริการผ่านเทคโนโลยีสื่อสาร และคอมพิวเตอร์สมัยใหม่ ศึกษาทักษะการออกแบบสื่อตามจุดมุ่งหมาย การเขียนโปรแกรมคอมพิวเตอร์เพื่อรวมสื่อ มัลติมีเดียกับอุปกรณ์สื่อสาร การผลิตภาพนิ่งและภาพเคลื่อนไหว การพัฒนาเนื้อหาและบริการผ่านเครือข่ายอินเทอร์เน็ต การพัฒนาเกม ประเภทต่างๆ การผลิตสื่อสร้างสรรค์เพื่อการเรียนรู้ด้วยตนเอง เทคโนโลยีที่เกี่ยวข้องกับสื่อ อาทิ การบันทึกสื่อ การลดขนาดสื่อมัลติมีเดีย เครือข่ายอินเทอร์เน็ต หลักการและการทำงานของเครือข่ายโทรศัพท์เคลื่อนที่ และโครงข่ายสื่อสารความเร็วสูง ฝึกฝนการผลิตและนำเสนอสื่อมัลติมีเดีย เช่น การควบคุมการผลิตภาพและเสียงใน สตูดิโอ การสร้างสรรค์เนื้อหาและบริการใหม่ๆ บนโทรศัพท์เคลื่อนที่ การผลิตและทดสอบเกมคอมพิวเตอร์ และการประยุกต์อุปกรณ์สมัยใหม่ร่วมกับสื่อมัลติมีเดีย
จุดเด่น ตลาดงานมีความต้องการงานทางด้าน Multimedia Engineer & Developer เป็นอย่างมาก ซึ่งภาควิชาฯ เน้นให้นักศึกษาได้รับการฝึกฝนทักษะในการพัฒนาสื่อมัลติมีเดียโดยใช้โปรแกรมประยุกต์ต่างๆ เช่น Macromedia flash, Adobe Premiere และ Adobe After Effect เป็นต้น โดยนักศึกษาสามารถเพิ่มพูนทักษะให้สูงขึ้น ในรายวิชาปฏิบัติการและการจัดทำโครงงานย่อยในรายวิชาบรรยายต่างๆ เช่น Creative Multimedia Skill, Multimedia Production, Multimedia Programming และ Multimedia Special Effect เป็นต้น สายวิชาอีกด้านหนึ่งที่ตลาดมีความต้องการคือ ด้านของ Computer Graphics & Art Designers ซึ่งนักศึกษาจะได้เรียนรู้การสร้างผลงานจากรายวิชา Computer Graphic and Animation และ Advanced Computer Graphics and Animation ซึ่งทั้งสองวิชานี้จะมีโครงงานประจำวิชาให้นักศึกษาได้ฝึกฝนเพื่อเสริมทักษะปฏิบัติ โดยการสร้างผลงานทางด้านนี้ จะเน้นให้สร้างสรรค์ผลงานบนพื้นฐานทางวิทยาศาสตร์ ซึ่งอาศัยความรู้ที่ได้จากวิชาฟิสิกส์ทำให้ผลงานที่ออกมานั้นมาความสมจริงและเป็นธรรมชาติมากยิ่งขึ้น 
หลักสูตรวิศวกรรมมัลติมีเดียและเอ็นเตอร์เทนเมนท์เป็น หลักสูตรที่เน้นการเรียนการสอนเพื่อสร้างนวัตกรรมและมัลติมีเดียสำหรับ Event & Exhibition & Concert เน้นพัฒนาสื่อมัลติมีเดียขั้นสูง ที่ต้องใช้องค์ความรู้ทางวิศวกรรมและการเขียนโปรแกรม นำเอาเทคโนโลยีทางด้าน CG & Animation ร่วมกับแสง สี เสียง มาสร้างเรื่องในงานแสดงและอีเว้นท์งานแสง สี เสียง ครบวงจร
ค่าเล่าเรียนทั้งหมดสำหรับหลักสูตรวิศวกรรมมัลติมีเดียและเอ็นเตอร์เทนเมนท์คือ 355,880 บาท
หลักสูตรวิศวกรรมไฟฟ้า : ก้าวเข้าสู่งานด้านการผลิตและการส่งจ่ายไฟฟ้า การออกแบบ การควบคุม และการรักษาคุณภาพและความเชื่อถือได้ของระบบไฟฟ้าทั้งในระดับโครงข่ายและภายในโรงงาน สำนักงาน และอาคารที่อยู่อาศัย การควบคุมเครื่องจักรกลไฟฟ้า และระบบควบคุมอัตโนมัติ เสริมความรู้ทางด้าน AutoCad, PLC, Renewable Energy, Building Management มีทักษะและความสามารถอย่างสมบูรณ์ เพื่อก้าวสู่การเป็นผู้พัฒนาทางด้านเทคโนโลยีระดับสากลในปัจจุบัน เรียนรู้ระบบต่างๆ ในห้องปฏิบัติการไฟฟ้าที่มีอุปกรณ์ที่เกี่ยวข้อง โดยลงมือปฏิบัติภายใต้การดูแลของผู้เชี่ยวชาญ ทั้งอาจารย์ภายในและอาจารย์พิเศษของหลักสูตรที่มาจากภาคอุตสาหกรรม ที่จะช่วยให้เข้าใจการทำงานพร้อมกับเรียนรู้ผ่านประสบการณ์ตรงของเหล่าอาจารย์ หลักสูตรวิศวกรรมไฟฟ้าเป็นหลักสูตรที่มีการสอนเน้นด้านการประกอบอาชีพทางวิชาชีพ การคิดอย่างเป็นระบบและการปฏิบัติจริง ถือเป็นสายงานที่ไม่เคยตกเทรนด์ ตอบโจทย์ตลาดงาน สามารถประกอบอาชีพได้หลากหลาย
หลักสูตรวิศวกรรมไฟฟ้าเน้นปฏิบัติควบคู่ความรู้ทฤษฎี มีอุปกรณ์และห้องปฏิบัติการที่พร้อมให้เรียนรู้ และเสริมองค์ความรู้ทางด้าน Renewable Energy และ Building Management ซึ่งเป็นที่ต้องการในตลาดงานปัจจุบัน เพิ่มโอกาสในการสอบใบประกอบวิชาชีพ (กว.)
ค่าเล่าเรียนทั้งหมดสำหรับหลักสูตรวิศวกรรมไฟฟ้าคือ 353,880 บาท
ในหลักสูตรวิศวกรรมศาสตร์บัณฑิต ได้มีการบรรจุวิชาการฝึกงานวิศวกรรม (Engineering Practices) ให้ลงทะเบียนเป็นวิชาเฉพาะบังคับ โดยกำหนดให้นักศึกษาเข้ารับการฝึกงานกับสถานประกอบการภายนอก เช่น ภาคธุรกิจและอุตสาหกรรมหรือหน่วยงานภายในที่เกี่ยวข้องโดยต้องได้รับความเห็นชอบจากคณาอาจารย์ที่ปรึกษาโครงการ โดยมีช่วงระยะเวลาฝึกงานประมาณ 2 เดือน หรือ อย่างน้อยเป็นเวลา 6 สัปดาห์ หรือประมาณ 180 ชั่วโมง ซึ่งจะมีคณาจารย์ที่รับผิดชอบโครงการดังกล่าวร่วมประสานงานในทุกขั้นตอน ตลอดจนการตรวจเยี่ยมสถานที่ฝึกงาน การประเมินผลฝึกงานและการจัดกิจกรรมต่างๆภายหลังการฝึกงาน
เรามีห้องปฏิบัติการสำหรับนักศึกษา 3 ห้อง ได้แก่ ห้องปฏิบัติการหุ่นยนต์ ห้องปฏิบัติการปัญญาประดิษฐ์ และ BU-Croccs ซึ่งเป็นสูนย์การเรียนรู้นานาชาติ ห้องปฏิบัติการเหล่านี้มีอุปกรณ์ครบครันและออกแบบมาเพื่อรองรับกิจกรรมการวิจัยและปฏิบัติที่หลากหลาย ช่วยให้นักศึกษาและนักวิจัยได้สำรวจและสร้างสรรค์ในสาขาของตน
ในระดับปริญญาโทของคณะวิศวกรรมศาสตร์มหาวิทยาลัยกรุงเทพ มีหลักสูตร Master of Engineering in Electrical & Computer Engineering เป็นหลักสูตร International Program มีเป้าหมายที่จะผลิตผู้ที่มีความรู้เชิงลึกเกี่ยวกับวิศวกรรมไฟฟ้าและคอมพิวเตอร์ นักศึกษาจะได้ทำงานร่วมกับทีมคณาจารย์และนักวิจัยของคณะวิศวกรรมศาสตร์ด้านการวิจัยที่ล้ำสมัย นักศึกษาจะได้รับพื้นฐานที่จำเป็นและการฝึกอบรมทักษะที่สำคัญ
ในระดับปริญญาเอกของคณะวิศวกรรมศาสตร์มหาวิทยาลัยกรุงเทพ มีหลักสูตร Doctor of Engineering in Electrical and Computer Engineering  เป็นหลักสูตร International Program ซึ่งมุ่งเน้นไปที่การทำโครงงานวิจัยที่ หรือการเขียนวิทยานิพนธ์ นักศึกษาจะได้ทำงานร่วมกับทีมคณาจารย์และนักวิจัยของคณะวิศวกรรมศาสตร์ด้านการวิจัยที่ล้ำสมัย นักศึกษาจะได้รับพื้นฐานที่จำเป็นและการฝึกอบรมทักษะที่สำคัญ

คุณจะต้องใช้ข้อมูลข้างต้นเพื่อตอบคำถามเท่านั้น ถ้าใช้ข้อมูลข้างต้นตอบคำถามไม่ได้ให้ตอบว่าฉันไม่รู้ คำตอบของคุณจะต้องสั้นโดยห้ามใช้ตัวอักษรเกิน 100 ตัวต่อการตอบ 1 ครั้ง
You must use the information above only, if you could not provide an answer from the information above please respond with you don't know the answer.
If you get the input in English you must response with english, If you get the input in Thai you must respon with Thai.
You can not response with more than 3 sentenses. Your response must be short.
if you get the input 'hmmm' please response with 'sorry i don't understand'.
'''

In [8]:
def create_session(context):
    print('calling create_session_function')
    aiplatform.init(project="noonmidday")
    chat_model = ChatModel.from_pretrained("chat-bison@001")
    parameters = {
        "temperature": 0.4,
        "max_output_tokens": 100,
        "top_k": 5,
        "top_p": 0.50,
        "context": context,
        ############# todo ##############
        # "examples": [
        #     InputOutputTextPair(
        #         input_text="I would like to reserve a room",
        #         output_text="Sure, I'd be happy to help you with that. What type of room would you like to reserve?",
        #     ),
        # ],
        ############# todo ##############
    }
    chat = ChatSession(model=chat_model, **parameters)
    return chat

# context = context.strip()
chat_model = create_session(thai_context)

calling create_session_function


In [19]:
def response(chat, message):
    print('calling response_function')
    start = time.time()
    response = chat.send_message(
        message=message, max_output_tokens=265, temperature=0.5, top_k=35, top_p=0.8
    )
    print("Process time: " + str((time.time() - start)))
    return response.text

tts_client = texttospeech.TextToSpeechClient()
def text_to_speech(text, language_code):
    print('calling text_to_speech_function')
    start = time.time()
    input_text = texttospeech.SynthesisInput(text=text)

    voice = texttospeech.VoiceSelectionParams(
        language_code=language_code,
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        pitch=0
    )
    response = tts_client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    #Save the audio response to a file
    with open("response.mp3", "wb") as out_file:
        out_file.write(response.audio_content)
    print("Process time: " + str((time.time() - start)))
    #Play the audio file
    playsound('response.mp3')
    print("Process time: " + str((time.time() - start)))

from google.cloud import speech
# Instantiates a client
speech_client = speech.SpeechClient()

def audio_to_text(filename):
    with open(filename, "rb") as audio_file:
        content = audio_file.read()
        
    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code='en-US',
        alternative_language_codes=['th-TH']
    )

    # Detects speech in the audio file
    response = speech_client.recognize(config=config, audio=audio)
    
    try:
        result = response.results[0].alternatives[0].transcript
    except:
        result = "hmmm"
    print(f"Transcript: {result}")

    return result

class Recorder():
    def __init__(self, filename):
        self.audio_format = pyaudio.paInt16
        self.channels = 1
        self.sample_rate = 44100
        self.chunk = int(0.03*self.sample_rate)
        self.filename = filename
        self.START_KEY = 'r'
        self.STOP_KEY = 's'
        self.QUIT_KEY = 'q'


    def record(self):
        recorded_data = []
        p = pyaudio.PyAudio()

        stream = p.open(format=self.audio_format, channels=self.channels,
                        rate=self.sample_rate, input=True,
                        frames_per_buffer=self.chunk)
        while(True):
            data = stream.read(self.chunk)
            recorded_data.append(data)
            if keyboard.is_pressed(self.STOP_KEY):
                print("Stop recording")
                # stop and close the stream
                stream.stop_stream()
                stream.close()
                p.terminate()
                #convert recorded data to numpy array
                recorded_data = [np.frombuffer(frame, dtype=np.int16) for frame in recorded_data]
                wav = np.concatenate(recorded_data, axis=0)
                wavfile.write(self.filename, self.sample_rate, wav)
                break


    def listen(self):
        print(f"Press `{self.START_KEY}` to start and `{self.STOP_KEY}` to stop!")
        while True:
            if keyboard.is_pressed(self.START_KEY):
                print('recording')
                self.record()
                break

def speech_to_text():
    print('calling speech_to_text_function')
    recorder = Recorder("record.wav") #name of output file
    recorder.listen()
    return audio_to_text("record.wav")

tt_client = translate.Client()
def chat_with_bot(chat_history):
    
    QUIT_KEY = 'q'
    print(f"Press {QUIT_KEY} to quit!")
    while keyboard.is_pressed('q') == False:
        user_input = speech_to_text()
        chat_response = response(chat_model,user_input)
        language = tt_client.detect_language(chat_response)
        text_to_speech(chat_response,language['language'])
        chat_history.append((user_input,chat_response))
        chat_with_bot(chat_history)

    return chat_history

In [20]:
######## run คำสั่งนี้เพื่อเริ่มคุย ########
chat_history = []
chat_with_bot(chat_history)

Press q to quit!
calling speech_to_text_function
Press `r` to start and `s` to stop!
recording
Stop recording
Transcript: สวัสดีค่ะ
calling response_function
Process time: 2.1951541900634766
calling text_to_speech_function
Process time: 0.7692205905914307
Process time: 6.551069259643555
Press q to quit!


[('สวัสดีค่ะ', 'สวัสดีค่ะ ยินดีต้อนรับสู่มหาวิทยาลัยกรุงเทพ มีอะไรให้ช่วยค่ะ')]